In [0]:
import sys
import numpy as np
import pandas as pd
from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/images_background.zip', 'r')
zip_ref.extractall('/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/')
zip_ref.close()

import zipfile
zip_ref = zipfile.ZipFile('/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/images_evaluation.zip', 'r')
zip_ref.extractall('/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/')
zip_ref.close()

In [0]:
train_folder = "/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/images_background/"
val_folder = "/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/images_evaluation"
save_path = '/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/'

In [0]:
def loadimgs(path,n = 0):
    '''
    path => Path of train directory or test directory
    '''
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    # we load every alphabet seperately so we can isolate them later
    for alphabet in os.listdir(path):
        print("loading alphabet: " + alphabet)
        lang_dict[alphabet] = [curr_y,None]
        alphabet_path = os.path.join(path,alphabet)
        # every letter/category has it's own column in the array, so  load seperately
        for letter in os.listdir(alphabet_path):
            cat_dict[curr_y] = (alphabet, letter)
            category_images=[]
            letter_path = os.path.join(alphabet_path, letter)
            # read all the images in the current category
            for filename in os.listdir(letter_path):
                image_path = os.path.join(letter_path, filename)
                image = imread(image_path)
                category_images.append(image)
                y.append(curr_y)
            try:
                X.append(np.stack(category_images))
            # edge case  - last one
            except ValueError as e:
                print(e)
                print("error - category_images:", category_images)
            curr_y += 1
            lang_dict[alphabet][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,lang_dict

In [0]:
X,y,c=loadimgs(train_folder)

loading alphabet: Alphabet_of_the_Magi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


KeyboardInterrupt: ignored

In [0]:
with open(os.path.join(save_path,"train.pickle"), "wb") as f:
    pickle.dump((X,c),f)

In [0]:
X,y,c=loadimgs(val_folder)
with open(os.path.join(save_path,"val.pickle"), "wb") as f:
	pickle.dump((X,c),f)

In [0]:
def initialize_weights(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [0]:
def initialize_bias(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)


In [0]:
weights = initialize_weights((1000,1))
bias = initialize_bias((1000,1))

In [0]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [0]:
model = get_siamese_model((105, 105, 1))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         38947648    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None

In [0]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [0]:
data_path = '/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6'

In [0]:
with open(os.path.join(data_path, "train.pickle"), "rb") as f:
    (X, classes) = pickle.load(f)

with open(os.path.join(data_path, "val.pickle"), "rb") as f:
    (Xval, val_classes) = pickle.load(f)
    
print("Training alphabets: \n")
print(list(classes.keys()))
print("Validation alphabets:", end="\n\n")
print(list(val_classes.keys()))

Training alphabets: 

['Alphabet_of_the_Magi', 'Anglo-Saxon_Futhorc', 'Arcadian', 'Armenian', 'Asomtavruli_(Georgian)', 'Balinese', 'Bengali', 'Blackfoot_(Canadian_Aboriginal_Syllabics)', 'Braille', 'Burmese_(Myanmar)', 'Cyrillic', 'Early_Aramaic', 'Futurama', 'Grantha', 'Greek', 'Gujarati', 'Hebrew', 'Inuktitut_(Canadian_Aboriginal_Syllabics)', 'Japanese_(hiragana)', 'Japanese_(katakana)', 'Korean', 'Latin', 'Malay_(Jawi_-_Arabic)', 'Mkhedruli_(Georgian)', 'N_Ko', 'Ojibwe_(Canadian_Aboriginal_Syllabics)', 'Sanskrit', 'Syriac_(Estrangelo)', 'Tagalog', 'Tifinagh']
Validation alphabets:

['Oriya', 'ULOG', 'Tengwar', 'Malayalam', 'Atlantean', 'Keble', 'Manipuri', 'Gurmukhi', 'Tibetan', 'Aurek-Besh', 'Ge_ez', 'Angelic', 'Old_Church_Slavonic_(Cyrillic)', 'Kannada', 'Avesta', 'Mongolian', 'Syriac_(Serto)', 'Atemayar_Qelisayer', 'Sylheti', 'Glagolitic']


In [0]:
test = np.load('/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/test.npy')

In [0]:
class Siamese_Loader:
    """For loading batches and testing tasks to a siamese net"""
    def __init__(self, path, data_subsets = ["train", "val"]):
        self.data = {}
        self.categories = {}
        self.info = {}
        
        for name in data_subsets:
            file_path = os.path.join(path, name + ".pickle")
            print("loading data from {}".format(file_path))
            with open(file_path,"rb") as f:
                (X,c) = pickle.load(f)
                self.data[name] = X
                self.categories[name] = c

    def get_batch(self,batch_size,s="train"):
        """Create batch of n pairs, half same class, half different class"""
        X=self.data[s]
        n_classes, n_examples, w, h = X.shape

        #randomly sample several classes to use in the batch
        categories = rng.choice(n_classes,size=(batch_size,),replace=False)
        #initialize 2 empty arrays for the input image batch
        pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
        #initialize vector for the targets, and make one half of it '1's, so 2nd half of batch has same class
        targets=np.zeros((batch_size,))
        targets[batch_size//2:] = 1
        for i in range(batch_size):
            category = categories[i]
            idx_1 = rng.randint(0, n_examples)
            pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
            idx_2 = rng.randint(0, n_examples)
            #pick images of same class for 1st half, different for 2nd
            if i >= batch_size // 2:
                category_2 = category  
            else: 
                #add a random number to the category modulo n classes to ensure 2nd image has
                # ..different category
                category_2 = (category + rng.randint(1,n_classes)) % n_classes
            pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
        return pairs, targets
    
    def generate(self, batch_size, s="train"):
        """a generator for batches, so model.fit_generator can be used. """
        while True:
            pairs, targets = self.get_batch(batch_size,s)
            yield (pairs, targets)    

    def make_oneshot_task(self,N,s="val",language=None):
        """Create pairs of test image, support set for testing N way one-shot learning. """
        X=self.data[s]
        n_classes, n_examples, w, h = X.shape
        indices = rng.randint(0,n_examples,size=(N,))
        if language is not None:
            low, high = self.categories[s][language]
            if N > high - low:
                raise ValueError("This language ({}) has less than {} letters".format(language, N))
            categories = rng.choice(range(low,high),size=(N,),replace=False)
            
        else:#if no language specified just pick a bunch of random letters
            categories = rng.choice(range(n_classes),size=(N,),replace=False)            
        true_category = categories[0]
        ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
        test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
        support_set = X[categories,indices,:,:]
        support_set[0,:,:] = X[true_category,ex2]
        support_set = support_set.reshape(N, w, h,1)
        targets = np.zeros((N,))
        targets[0] = 1
        targets, test_image, support_set = shuffle(targets, test_image, support_set)
        pairs = [test_image,support_set]

        return pairs, targets
    
    def test_oneshot(self,model,N,k,s="val",verbose=0):
        """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
        n_correct = 0
        if verbose:
            print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
        for i in range(k):
            inputs, targets = self.make_oneshot_task(N,s)
            probs = model.predict(inputs)
            if np.argmax(probs) == np.argmax(targets):
                n_correct+=1
        percent_correct = (100.0*n_correct / k)
        if verbose:
            print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
        return percent_correct
    
    def train(self, model, epochs, verbosity):
        model.fit_generator(self.generate(batch_size))
    

loader = Siamese_Loader(data_path)

loading data from /content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/train.pickle
loading data from /content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/val.pickle


In [0]:
evaluate_every = 10 # interval for evaluating on one-shot tasks
loss_every = 20 # interval for printing loss (iterations)
batch_size = 32
n_iter = 20000
N_way = 20 # how many classes for testing one-shot tasks>
n_val = 250 # how many one-shot tasks to validate on?
best = -1
weights_path_2 = os.path.join(data_path, "model_weights.h5")
#model_path = '/content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/weights'
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter):
    (inputs,targets)=loader.get_batch(batch_size)
    loss=model.train_on_batch(inputs,targets)
    print("\n ------------- \n")
    print("Loss: {0}".format(loss)) 
    if i % evaluate_every == 0:
        print("Time for {0} iterations: {1}".format(i, time.time()-t_start))
        val_acc = loader.test_oneshot(model,N_way,n_val,verbose=True)
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            print("Saving weights to: {0} \n".format(weights_path_2))
            model.save_weights(weights_path_2)
            best=val_acc
    
    if i % loss_every == 0:
        print("iteration {}, training loss: {:.2f},".format(i,loss))

        

model.load_weights(weights_path_2)

Starting training process!
-------------------------------------

 ------------- 

Loss: 4.04870080947876

 ------------- 

Loss: 3.9655749797821045

 ------------- 

Loss: 4.020485877990723

 ------------- 

Loss: 3.9704413414001465

 ------------- 

Loss: 4.004826545715332

 ------------- 

Loss: 3.907961130142212

 ------------- 

Loss: 3.987091541290283

 ------------- 

Loss: 3.987914800643921

 ------------- 

Loss: 3.93373966217041

 ------------- 

Loss: 3.8957748413085938
Time for 10 iterations: 1.9074890613555908
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 29.2% 20 way one-shot learning accuracy 

Current best: 29.2, previous best: -1
Saving weights to: /content/gdrive/My Drive/Neural Net & Fuzzy Logic/Lab/Lab6/model_weights.h5 


 ------------- 

Loss: 3.845153570175171

 ------------- 

Loss: 3.894932746887207

 ------------- 

Loss: 3.804736375808716

 ------------- 

Loss: 3.9074349403381348

 ------------- 

Loss: 3.9152960777282

In [0]:
test.shape

(200, 2, 10, 105, 105, 1)

In [0]:
weights_path_2 = os.path.join(data_path, "model_weights.h5")
model.load_weights(weights_path_2)

In [0]:
pairs.shape

NameError: ignored

In [0]:
model.predict([test[0][0].reshape(10, 105, 105,1),test[0][1].reshape(10, 105, 105,1)])

array([[1.8179417e-06],
       [6.6858232e-03],
       [8.0214930e-01],
       [7.3972344e-04],
       [4.7618151e-03],
       [8.4581459e-01],
       [2.4440885e-04],
       [4.1723251e-07],
       [1.0542751e-03],
       [6.2389866e-02]], dtype=float32)

In [0]:
sol = np.zeros(shape = (200,2))
for i in range(len(test)):
    maxi = 0
    p = model.predict([test[i][0].reshape(10, 105, 105,1),test[i][1].reshape(10, 105, 105,1)])
    maxim = p[0]
    for j in range(len(p)):
        if p[j]>maxim:
            maxi = j
            maxim = p[j]
    
    sol[i][0] = i
    sol[i][1] = maxi

    

In [0]:
sol = sol.astype(dtype = 'int64')

In [0]:
sol

array([[  0,   5],
       [  1,   5],
       [  2,   8],
       [  3,   8],
       [  4,   0],
       [  5,   3],
       [  6,   9],
       [  7,   4],
       [  8,   3],
       [  9,   5],
       [ 10,   9],
       [ 11,   9],
       [ 12,   6],
       [ 13,   2],
       [ 14,   8],
       [ 15,   8],
       [ 16,   9],
       [ 17,   1],
       [ 18,   3],
       [ 19,   1],
       [ 20,   2],
       [ 21,   1],
       [ 22,   5],
       [ 23,   3],
       [ 24,   2],
       [ 25,   4],
       [ 26,   5],
       [ 27,   3],
       [ 28,   5],
       [ 29,   8],
       [ 30,   6],
       [ 31,   2],
       [ 32,   7],
       [ 33,   4],
       [ 34,   1],
       [ 35,   9],
       [ 36,   5],
       [ 37,   5],
       [ 38,   1],
       [ 39,   9],
       [ 40,   6],
       [ 41,   0],
       [ 42,   6],
       [ 43,   4],
       [ 44,   4],
       [ 45,   4],
       [ 46,   5],
       [ 47,   7],
       [ 48,   0],
       [ 49,   4],
       [ 50,   0],
       [ 51,   0],
       [ 52,

In [0]:
subm = pd.DataFrame(sol)

In [0]:
subm = subm.rename(index=str, columns={0:'id' , 1: "Category"})

In [0]:
subm.to_csv('Submission.csv', index=False)